# IPOTESI DI PRIMA BOZZA PER REGOLA 

In [24]:
import pandas as pd

# Lettura del file Excel
building_type = pd.read_excel(
    "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/number_buildings_type.xlsx"
)

# Dizionario dei tipi di edificio
type_dict = {
    "religious_type": [
        "religious",
        "cathedral",
        "chapel",
        "church",
        "kingdom_hall",
        "monastery",
        "mosque",
        "presbytery",
        "shrine",
        "synagogue",
        "temple",
    ],
    "accomodation_type": [
        "apartments",
        "barracks",
        "bungalow",
        "cabin",
        "detached",
        "dormitory",
        "farm",
        "ger",
        "hotel",
        "house",
        "houseboat",
        "residential",
        "semidetached_house",
        "static_caravan",
        "stilt_house",
        "terrace",
        "tree_house",
        "trullo",
    ],
    "commercial_type": [
        "commercial",
        "industrial",
        "kiosk",
        "office",
        "retail",
        "supermarket",
        "warehouse",
    ],
    "civic_type": [
        "bakehouse",
        "bridge",
        "civic",
        "college",
        "fire_station",
        "government",
        "gatehouse",
        "hospital",
        "kindergarten",
        "museum",
        "public",
        "school",
        "toilets",
        "train_station",
        "transportation",
        "university",
    ],
    "agriculture_type": [
        "barn",
        "conservatory",
        "cowshed",
        "farm_auxiliary",
        "greenhouse",
        "slurry_tank",
        "stable",
        "sty",
        "livestock",
    ],
    "sport_type": [
        "grandstand",
        "pavilion",
        "riding_hall",
        "sports_hall",
        "sports_centre",
        "stadium",
    ],
    "cars_type": ["carport", "garage", "garages", "parking"],
    "technical_type": [
        "digester",
        "service",
        "tech_cab",
        "transformer_tower",
        "water_tower",
        "storage_tank",
        "silo",
    ],
    "other_type": [
        "beach_hut",
        "bunker",
        "castle",
        "construction",
        "container",
        "guardhouse",
        "military",
        "outbuilding",
        "pagoda",
        "quonset_hut",
        "roof",
        "tent",
        "tower",
        "windmill",
        "yes",
        "user defined",
    ],
}

# Creazione dei DataFrame vuoti
dataframes = {}
for key, value in type_dict.items():
    df = pd.DataFrame(index=value)
    # Aggiunta delle colonne per i cluster
    for cluster in building_type["cluster"].unique():
        df[f"cluster_{cluster}"] = 0
    dataframes[key] = df


# Funzione per aggiornare i DataFrame per ogni cluster separatamente
def update_dataframes_by_cluster(main_df, dataframes):
    clusters = main_df["cluster"].unique()

    for cluster in clusters:
        cluster_df = main_df[main_df["cluster"] == cluster]

        for _, row in cluster_df.iterrows():
            for column in main_df.columns[1:]:  # Ignorare la colonna 'cluster'
                value = row[column]
                if value > 0:
                    for df in dataframes.values():
                        if column in df.index:
                            df.at[column, f"cluster_{cluster}"] += value


# Aggiornamento dei DataFrame per ogni cluster separatamente
update_dataframes_by_cluster(building_type, dataframes)

# Concatenazione dei DataFrame in un unico DataFrame per tipo di edificio
concatenated_dfs = {}
for key, df in dataframes.items():
    concatenated_dfs[key] = df
output_path = "C:/Users/margh/Desktop/pypsa/pypsa-distribution/resources/buildings/"
for key, df in concatenated_dfs.items():
    df.to_csv(f"{output_path}_{key}.csv", index=True)